# Image Classifier: Poison Ivy
 Trains a model to classify a plant image as a type of poison ivy (or not).

Below we do the following:
1. Setup the training environment.
2. Load images of different poison ivy plants and look-alikes
3. Train an image classifier model.
3. Convert the model to CoreML and upload it to Skafos.

## Environment Setup
Below we ensure `CUDA 10` is installed and then use pip to install `turicreate`, `mxnet-cu100`, and `skafos` libraries.

In [ ]:
# Confirm that you have CUDA 10
!nvcc --version

In [ ]:
# Install libraries - you might need to restart the runtime after doing this
!pip install turicreate==5.4
# The wrong version of mxnet will be installed
!pip uninstall -y mxnet
# Install CUDA10-compatible version of mxnet
!pip install mxnet-cu100
# install Skafos python sdk
!pip install skafos

## Data Preparation and Model Training
The training data for this example are images of various plant species (some poisonous, some not), gathered and labeled by hand. One of the limitations of this space is having enough training data. Because our data is limited, if we were to try to make a more accurate model, we would need to collect and label more images for each class.

After unzipping and extracting the images, they are loaded into a Turi Create SFrame and labels are created for each image based on the path. The data is randomly split into train and test sets, where 80% of the data is used for training and 20% is used for model evaluation (if you desire). Training this model with a GPU is much faster than CPU time. By default, this runtime environment should be using a Python 3 GPU backend instance. Below, we tell Turicreate to use all available GPUs for processing.

In [ ]:
# Import libraries and tell Turicreate to use all GPUs available - this may throw a warning
import urllib
import tarfile
import os

import coremltools
import turicreate as tc
tc.config.set_num_gpus(-1)

In [ ]:
# Specify the data set download url
data_url = "https://s3.amazonaws.com/skafos.example.data/ImageClassifier/poisonPlants.tar.gz"
data_path = "poisonPlants.tar.gz"

# Pull the compressed data and extract it
retrieve = urllib.request.urlretrieve(data_url, data_path)
tar = tarfile.open(data_path)
tar.extractall()
tar.close()

In [ ]:
# Load images - you can ignore various jpeg decode errors
data = tc.image_analysis.load_images('poisonPlants', with_path=True, ignore_failure=True)

# From the path-name, create a label column. This labels each image as the appropriate plant
data['label'] = data['path'].apply(lambda path: os.path.basename(os.path.dirname(path)))

In [ ]:
# Make a train-test split
train_data, test_data = data.random_split(0.8)

In [ ]:
# Train an image classification model - consider increasing max_iterations
model = tc.image_classifier.create(
    dataset=train_data,
    target='label',
    model='resnet-50',
    batch_size=4,
    max_iterations=10
)

# Image Classification Training Docs:
# https://apple.github.io/turicreate/docs/api/generated/turicreate.image_classifier.create.html#turicreate.image_classifier.create

## Model Evaluation

In [ ]:
# Let's see how the model performs on the hold out test data
predictions = model.predict(test_data)
accuracy = tc.evaluation.accuracy(test_data['label'], predictions)
print(f"Image classifier is {accuracy*100} % accurate on the testing dataset", flush=True)

## Model Export and Skafos Upload
- Convert the model to CoreML format so that it can run on an iOS device. Then deliver the model to your apps with **[Skafos](https://skafos.ai)**.

- If you don't already have an account, Sign Up for one **[here](https://dashboard.skafos.ai)**. 
- Once you've signed up for an account, grab an API token from your account settings.

In [ ]:
# Specify the CoreML model name
model_name = 'ImageClassifier'
coreml_model_name = model_name + '.mlmodel'

# Export the trained model to CoreML format
res = model.export_coreml(coreml_model_name)

In [ ]:
import skafos
from skafos import models
import os

# Set your API Token first for repeated use
os.environ["SKAFOS_API_TOKEN"] = "<YOUR-SKAFOS-API-TOKEN>"
skafos.summary()

In [ ]:
# You can retrieve this info with skafos.summary()
org_name = "<YOUR-SKAFOS-ORG-NAME>"    # Example: "mike-gmail-com-467h2"
app_name = "<YOUR-SKAFOS-APP-NAME>"    # Example: "ImageClassification-App"
model_name = "<YOUR-MODEL-NAME>"       # Example: "ImageClassificationModel"

# Upload model version to Skafos
model_upload_result = models.upload_version(
    files="ImageClassifier.mlmodel",
    org_name=org_name,
    app_name=app_name,
    model_name=model_name
)